In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
np.random.seed(420)
sys.path.append("/home/moritz/repositories/med_leak/")

In [3]:
def get_info(df):
    n_images = len(df)
    n_patients = df.patient_id.nunique()
    return n_images, n_patients

In [4]:
cxr_df = pd.read_csv('/home/moritz/data/chexpert/CheXpert-v1.0/train.csv')
cxp_demo = pd.read_csv('/home/moritz/data/chexpert/CheXpert-v1.0/CHEXPERT_DEMO.csv')

In [5]:
def process_main(df):
    df.fillna(0, inplace=True)
    if "path" not in df.columns:
        df.rename(columns={'Path': 'path'}, inplace=True)
    df['patient_id']= df['path']
    df['patient_id'] = df['patient_id'].map(lambda x: str(x).split("/")[2][7:])
    df['study_id'] = df['path']
    df['study_id'] = df['study_id'].map(lambda x: str(x).split("/")[3][5:])
    return df
    
def process_demo(df):
    if "patient_id" not in df.columns:
        df.rename(columns={'PATIENT': 'patient_id'}, inplace=True)
    df['patient_id'] = df['patient_id'].map(lambda x: str(x)[7:])
    df = df.rename(columns={'PRIMARY_RACE': 'race'})
    df = df.rename(columns={'PATIENT': 'patient_id'})
    df = df.rename(columns={'GENDER': 'sex'})
    df = df.rename(columns={'AGE_AT_CXR': 'age'})
    df = df.rename(columns={'ETHNICITY': 'ethnicity'})
    df = df.drop(['sex', 'age'], axis=1)
    return df
cxr_df = process_main(cxr_df)
cxp_demo = process_demo(cxp_demo)
cxr_df = cxr_df[cxr_df["Sex"] != "Unknown"]

In [6]:
n_original_images, n_original_patients = get_info(cxr_df)
f"{n_original_images} images from {n_original_patients} patients"

'223413 images from 64539 patients'

In [7]:
cxr_df = pd.merge(cxr_df, cxp_demo, on='patient_id', how='left')
assert get_info(cxr_df) == (n_original_images, n_original_patients), f"Number of images and patients should be the same: {get_info(cxr_df)} vs {(n_original_images, n_original_patients)}"

In [8]:
cxr_df.head()

,path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,...,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,patient_id,study_id,race,ethnicity
0,CheXpert-v1.0/train/patient00001/study1/view1_...,Female,68,Frontal,AP,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,00001,1,Other,Non-Hispanic/Non-Latino
1,CheXpert-v1.0/train/patient00002/study2/view1_...,Female,87,Frontal,AP,0.0,0.0,-1.0,1.0,0.0,...,-1.0,0.0,-1.0,0.0,1.0,0.0,00002,2,"White, non-Hispanic",Non-Hispanic/Non-Latino
2,CheXpert-v1.0/train/patient00002/study1/view1_...,Female,83,Frontal,AP,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,00002,1,"White, non-Hispanic",Non-Hispanic/Non-Latino
3,CheXpert-v1.0/train/patient00002/study1/view2_...,Female,83,Lateral,0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,00002,1,"White, non-Hispanic",Non-Hispanic/Non-Latino
4,CheXpert-v1.0/train/patient00003/study1/view1_...,Male,41,Frontal,AP,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,00003,1,"White, non-Hispanic",Non-Hispanic/Non-Latino


In [9]:
white = "White"
asian = "Asian"
black = "Black"
unknown = "Other/Unknown"

mask = cxr_df.race.str.contains("Black", na=False)
cxr_df.loc[mask, "race"] = black
mask = cxr_df.race.str.contains("White", na=False)
cxr_df.loc[mask, "race"] = white
mask = cxr_df.race.str.contains("Asian", na=False)
cxr_df.loc[mask, "race"] = asian

other_vals = [
    val for val in cxr_df["race"].unique() if val not in [white, asian, black]
]
cxr_df["race"].replace({val: "Other/Unknown" for val in other_vals}, inplace=True)
assert get_info(cxr_df) == (
    n_original_images,
    n_original_patients,
), f"Number of images and patients should be the same: {get_info(cxr_df)} vs {get_info(cxp_df)}"

/tmp/ipykernel_1269908/1553176884.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cxr_df["race"].replace({val: "Other/Unknown" for val in other_vals}, inplace=True)


In [10]:
cxr_df.Sex.value_counts(), cxr_df.race.value_counts()

(Sex
 Male      132636
 Female     90777
 Name: count, dtype: int64,
 race
 White            125491
 Other/Unknown     62689
 Asian             23272
 Black             11961
 Name: count, dtype: int64)

In [11]:
cxr_df.head()

,path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,...,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,patient_id,study_id,race,ethnicity
0,CheXpert-v1.0/train/patient00001/study1/view1_...,Female,68,Frontal,AP,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,00001,1,Other/Unknown,Non-Hispanic/Non-Latino
1,CheXpert-v1.0/train/patient00002/study2/view1_...,Female,87,Frontal,AP,0.0,0.0,-1.0,1.0,0.0,...,-1.0,0.0,-1.0,0.0,1.0,0.0,00002,2,White,Non-Hispanic/Non-Latino
2,CheXpert-v1.0/train/patient00002/study1/view1_...,Female,83,Frontal,AP,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,00002,1,White,Non-Hispanic/Non-Latino
3,CheXpert-v1.0/train/patient00002/study1/view2_...,Female,83,Lateral,0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,00002,1,White,Non-Hispanic/Non-Latino
4,CheXpert-v1.0/train/patient00003/study1/view1_...,Male,41,Frontal,AP,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,00003,1,White,Non-Hispanic/Non-Latino


In [12]:
#cxr_df.to_csv("/home/moritz/repositories/med_leak/data/csv/chexpert_train.csv", index=False)